In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
from selenium.webdriver.common.by import By
import pandas as pd

import numpy as np

In [11]:
product_dict = {
                # "4550583676513": "婦人　さらっと綿　汗取りパッド付きタンクトップ", # women
                # "4550584089855": "婦人　ストレッチキャミソール", ###########
                # "4550583675790": "婦人　さらっと綿　汗取りパッド付きキャミソール",
                # "4550583459871": "婦人　さらっと綿　汗取りパッド付きフレンチスリーブＴシャツ",
                # ############################
                # "4550583187330": "あったか綿　ウール　ハイネック長袖Ｔシャツ（２０２３年仕様）",
                # "4550583183172": "あったか綿　厚手　Ｕネック八分袖Ｔシャツ（２０２３年仕様）",
                # "4550583130084": "あったか綿　Ｕネック八分袖Ｔシャツ（２０２３年仕様）",
                # "4550583187231": "あったか綿　ウール　クルーネック長袖Ｔシャツ（２０２３年仕様）",  ###############
                ############################
                "4550583070205": "あったか綿　クルーネック長袖Ｔシャツ（２０２３年仕様）", # men'
                "4550583071776": "あったか綿　厚手　クルーネック長袖Ｔシャツ（２０２３年仕様）",
                "4550583065485": "あったか綿　クルーネック半袖Ｔシャツ（２０２３年仕様）",
                # "4550583067007": "あったか綿　Ｖネック半袖Ｔシャツ（２０２３年仕様）",
                ############################
                "4550583468316": "紳士　さらっと綿　クルーネック半袖Ｔシャツ",
                "4550583468132": "紳士　さらっと綿　Ｖネック半袖Ｔシャツ",
                "4550583468538": "紳士　さらっと綿　タンクトップ",


                }

In [12]:
gender = "男性"

for product_id in product_dict.keys():

    driver = webdriver.Chrome()
    driver.get(f"https://www.muji.com/jp/ja/store/review/cmdty/{product_id}")


    while True:
        time.sleep(2)

        try:
            elem = driver.find_element(By.XPATH,'//*[@id="contentId"]/div[1]/div[4]/div[2]/div/div/div/div[2]/div/div/div/span')
            elem.click()
        except:

            break

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    
    reviews = soup.find_all('div', class_='reviewList__ReviewlistStyled-sc-m4x8te-1 jBEBCp')
    stars = soup.find_all('ul', class_='ant-rate ant-rate-disabled rate__StyledRate-sc-1ntrhud-0 fmRmmu')
    date = soup.find_all('div', class_='reviewList__StyledCreatedDate-sc-m4x8te-20 cgMlEB created-date')
    detail = soup.find_all('div', class_='reviewList__StyledReview-sc-m4x8te-14 ctEpYw')
    product = soup.find_all('h4', class_='itemInfo__StyledItemName-sc-16ax6d7-4 XYnXg item-name')
    product = product_dict[product_id]

    gender_list = []
    age_list = []

    for d in detail:
        soup_d = BeautifulSoup(str(d), "html.parser")
        spans = soup_d.find_all('span')

        pres_gend_list_len = len(gender_list)
        pres_age_list_len = len(age_list)
        
        for span in spans:
            text = span.get_text()
            if '女性' in text or '男性' in text:
                gender_list.append(gender)

            if '代' in text:
                age = text.strip()
                age_list.append(age)

        if pres_gend_list_len == len(gender_list):
            gender_list.append(np.nan)

        if pres_age_list_len == len(age_list):
            age_list.append(np.nan)


    date_list = [d.get_text() for d in date]


    # starsの要素に"ant-rate-star ant-rate-star-full"が何回含まれるかカウントして、それをリストに格納する
    star_list = []
    for s in stars:
        single_rev_stars = s.find_all('li')
        single_rev_stars_count = sum(1 for star in single_rev_stars if "ant-rate-star ant-rate-star-full" in str(star))
        star_list.append(single_rev_stars_count)

    star_list = star_list[1:]

    review_list = [t.get_text() for t in reviews]
    # review_list = [t for t in review_list if "もっと見る購入サイズ" in t]
    review_listじ

    column_dict = {"product": product ,
                "review": review_list,
                "star": star_list,
                "gender": gender_list,
                "age": age_list,
                "date": date_list
                }

    df = pd.DataFrame(column_dict)




    df.to_csv(f'../data/muji/muji_{product}.csv', index=False)

In [8]:
product[0].get_text()

IndexError: list index out of range

In [9]:
product

[]

In [8]:
text

'件のコメント'

In [81]:
df

,product,review,star,gender,age,date
0,婦人さらっと綿汗取りパッド付きタンクトップ,その名の通りサラッとした綿で、汗を吸ってもベタつかず快適です。,4,女性,/50代,2024/07/10
1,婦人さらっと綿汗取りパッド付きタンクトップ,汗をよく吸うので夏でも気持ちがいいですが、一日着ると全体的に伸びてしまうのが難点です。\nな...,1,NaN,NaN,2024/07/10
2,婦人さらっと綿汗取りパッド付きタンクトップ,真夏には少し厚いと感じますが汗をちゃんと吸い乾きやすいので着ていて気持ちが良いです。少し小さ...,5,NaN,NaN,2024/07/07
3,婦人さらっと綿汗取りパッド付きタンクトップ,汗とりのタンクトップは脇のパット部分の位置が重要で、この商品は私にとって絶妙な位置にきました...,5,女性,/40代,2024/07/07
4,婦人さらっと綿汗取りパッド付きタンクトップ,今年に入ってから3枚目の購入です。そのくらい気に入ってます。生地は薄くて、縫い目の凹凸が少な...,4,女性,/50代,2024/07/07
...,...,...,...,...,...,...
79,婦人さらっと綿汗取りパッド付きタンクトップ,着心地は綿素材なので良いです。\n\n襟ぐりが何とも絶妙な空き具合！\n洋服によっては隠すこ...,5,女性,/30代,2024/04/10
80,婦人さらっと綿汗取りパッド付きタンクトップ,去年から着始めてとってもいい！\n肌触りもいいし、やっぱり脇汗が吸収してくれるのがついてるの...,5,女性,/40代,2024/04/09
81,婦人さらっと綿汗取りパッド付きタンクトップ,春夏シーズンで最高！何着もかいました。ただし値上げしたのでもう少し手頃な価格なら幸いです。,4,女性,/40代,2024/03/31
82,婦人さらっと綿汗取りパッド付きタンクトップ,さらっとしてて、着心地がとても良いです。高身長の私でも、ちょうど良いきたけで、ボトムスにイン...,5,女性,/40代,2024/03/28
